# **Bitcoin price prediction with PySpark - Data crawling**
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it



---


Description: data crawling on Bitcoin blochckain by querying Blockchain.com website.

# Global constants, dependencies, libraries and tools

In [48]:
# Main constants
LOCAL_RUNNING = True
MAIN_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [49]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(MAIN_DIR, force_remount=True)

Mounted at /content/drive


In [50]:
# Set main dir
MAIN_DIR = MAIN_DIR + "" if LOCAL_RUNNING else MAIN_DIR + "/MyDrive/BDC/project"

# Datasets dir
DATASET_RAW_DIR = MAIN_DIR + "/datasets/raw"

# Datasets name
DATASET_NAME = "bitcoin_blockchain_data_15min"

# Datasets path
DATASET_RAW = DATASET_RAW_DIR + "/" + DATASET_NAME + ".parquet"

In [51]:
# Useful imports
import pandas as pd
import functools

if not LOCAL_RUNNING: from google.colab import drive

from datetime import datetime, timedelta

# Metrics and parameters
I chose to collect data on the Bitcoin blockchain using the API of the website Blockchain.org, the most relevant information was retrieved from the year 2016 to the present day (a period for which there were moments of high volatility but also a lot of price lateralization).

The features taken under consideration were divided into several categories:

**Currency Statistics**

- **market-price:** the average USD market price across major bitcoin exchanges.
- **trade-volume:** the total USD value of trading volume on major bitcoin exchanges.
- **total-bitcoins:** the total number of mined bitcoin that are currently circulating on the network.
- **market-cap:** the total USD value of bitcoin in circulation.

**Block Details**

- **blocks-size:** the total size of the blockchain minus database indexes in megabytes.
- **avg-block-size:** the average block size over the past 24 hours in megabytes.
- **n-transactions-total:** the total number of transactions on the blockchain.
- **n-transactions-per-block:** the average number of transactions per block over the past 24 hours.

**Mining Information**

- **hash-rate:** the estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
- **difficulty:** a relative measure of how difficult it is to mine a new block for the blockchain.
- **miners-revenue:** total value of coinbase block rewards and transaction fees paid to miners.
- **transaction-fees-usd:** the total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

**Network Activity**

- **n-unique-addresses:** the total number of unique addresses used on the blockchain.
- **n-transactions:** the total number of confirmed transactions per day.
- **estimated-transaction-volume-usd:** the total estimated value in USD of transactions on the blockchain.

In [52]:
# Define the parameters
timespan = "4years" # Duration of the data (it was necessary to define it since it is possible to make requests for up to 6 years)
# Get today's date
end_date = datetime.today().strftime('%Y-%m-%d')
# Get the starting date
start_date = (datetime.today() - timedelta(days=365*4)).strftime('%Y-%m-%d')

# Metrics considered
metrics = [
          # Currency Statistics
          "market-price",
          "trade-volume",

          # Block Details
          "blocks-size",
          "avg-block-size",
          "n-transactions-total",
          "n-transactions-per-block",

          # Mining Information
          "hash-rate",
          "difficulty",
          "miners-revenue",
          "transaction-fees-usd",

          # Network Activity
          "n-unique-addresses",
          "n-transactions",
          "estimated-transaction-volume-usd"
]

# Data crawling

In [53]:
def data_crawler(timespan, metrics, start_date, end_date):
    # API info
    url = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'

    # Obtain data
    data = pd.read_csv(url, names=['timestamp', metrics])

    # Transform "timestamp" to datetime type
    data['timestamp'] = pd.to_datetime(data["timestamp"])

    # Select data up to the end date
    data = data[(data['timestamp'] < end_date)]

    return data

In [54]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain blockchain bata from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2019-09-23,10033.05,5.950181e+07,241221.519675,1.243720,457899079,2527.622807,6.738365e+07,1.189059e+13,1.464718e+07,3.220338e+05,544681.0,288149.0,8.550738e+08
1,2019-09-24,9683.38,1.200016e+08,241363.197868,1.163618,458186764,2250.082803,9.280029e+07,1.189059e+13,1.922035e+07,3.687073e+05,596954.0,353263.0,3.139283e+09
2,2019-09-25,8553.61,4.914318e+08,241545.985674,1.104534,458540193,2229.147436,9.220921e+07,1.189059e+13,1.722270e+07,3.652339e+05,571599.0,347747.0,2.087385e+09
3,2019-09-26,8432.23,2.690300e+08,241718.046369,0.970738,458888070,2080.555556,1.021557e+08,1.201767e+13,1.814204e+07,2.732286e+05,554702.0,355775.0,1.313285e+09
4,2019-09-27,8055.64,2.863395e+08,241883.747626,1.095424,459242846,2302.295455,8.372682e+07,1.275982e+13,1.373193e+07,2.806025e+05,532292.0,303903.0,1.158430e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,2023-09-17,26568.65,3.256705e+07,511582.358862,1.722461,895646989,4102.147651,4.010809e+08,5.415014e+13,2.547981e+07,7.516007e+05,854606.0,611220.0,8.873346e+08
1449,2023-09-18,26536.54,2.612496e+07,511839.128976,1.743559,896259253,4075.929487,4.199236e+08,5.415014e+13,2.776008e+07,1.269166e+06,852484.0,635845.0,2.336440e+09
1450,2023-09-19,26763.76,1.483430e+08,512111.211644,1.671013,896896158,3553.143885,3.856776e+08,5.581661e+13,2.510024e+07,1.282769e+06,741822.0,493887.0,2.555774e+09
1451,2023-09-20,27222.12,1.424000e+08,512343.247832,1.656908,897387893,3779.335616,4.145589e+08,5.711987e+13,2.606572e+07,1.286585e+06,823616.0,551783.0,2.181950e+09


In [55]:
# Check duplicated rows
len(df1['timestamp'].unique())

1453

In [56]:
# Retrieving market capitalization and total circulating data
metrics = [
          # Currency Statistics
          "total-bitcoins",                      # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
          "market-cap",                          # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2019-09-23 00:04:26,17952050.00,1.799873e+11
1,2019-09-24 04:54:49,17953837.50,1.746190e+11
2,2019-09-25 03:00:51,17955612.50,1.545978e+11
3,2019-09-26 00:04:16,17957400.00,1.513809e+11
4,2019-09-26 19:48:08,17959175.00,1.453526e+11
...,...,...,...
1498,2023-09-17 11:23:43,19488275.00,5.186220e+11
1499,2023-09-18 07:48:16,19489162.50,5.208674e+11
1500,2023-09-19 08:14:03,19490050.00,5.239900e+11
1501,2023-09-20 09:04:17,19490931.25,5.303677e+11


In [57]:
# Check duplicated rows
len(df2['timestamp'].unique())

1503

In [58]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)
df2

,timestamp,total-bitcoins,market-cap
0,2019-09-23,17952050.00,1.799873e+11
1,2019-09-24,17953837.50,1.746190e+11
2,2019-09-25,17955612.50,1.545978e+11
4,2019-09-26,17959175.00,1.453526e+11
5,2019-09-27,17960950.00,1.444420e+11
...,...,...,...
1498,2023-09-17,19488275.00,5.186220e+11
1499,2023-09-18,19489162.50,5.208674e+11
1500,2023-09-19,19490050.00,5.239900e+11
1501,2023-09-20,19490931.25,5.303677e+11


In [59]:
# Check duplicated rows
len(df2['timestamp'].unique())

1419

In [60]:
# Add the market capitalization and total circulating data to the main dataset
all_data = pd.merge(df1, df2, how="inner", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2019-09-23,10033.05,5.950181e+07,241221.519675,1.243720,457899079,2527.622807,6.738365e+07,1.189059e+13,1.464718e+07,3.220338e+05,544681.0,288149.0,8.550738e+08,17952050.00,1.799873e+11
1,2019-09-24,9683.38,1.200016e+08,241363.197868,1.163618,458186764,2250.082803,9.280029e+07,1.189059e+13,1.922035e+07,3.687073e+05,596954.0,353263.0,3.139283e+09,17953837.50,1.746190e+11
2,2019-09-25,8553.61,4.914318e+08,241545.985674,1.104534,458540193,2229.147436,9.220921e+07,1.189059e+13,1.722270e+07,3.652339e+05,571599.0,347747.0,2.087385e+09,17955612.50,1.545978e+11
3,2019-09-26,8432.23,2.690300e+08,241718.046369,0.970738,458888070,2080.555556,1.021557e+08,1.201767e+13,1.814204e+07,2.732286e+05,554702.0,355775.0,1.313285e+09,17959175.00,1.453526e+11
4,2019-09-27,8055.64,2.863395e+08,241883.747626,1.095424,459242846,2302.295455,8.372682e+07,1.275982e+13,1.373193e+07,2.806025e+05,532292.0,303903.0,1.158430e+09,17960950.00,1.444420e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,2023-09-17,26568.65,3.256705e+07,511582.358862,1.722461,895646989,4102.147651,4.010809e+08,5.415014e+13,2.547981e+07,7.516007e+05,854606.0,611220.0,8.873346e+08,19488275.00,5.186220e+11
1408,2023-09-18,26536.54,2.612496e+07,511839.128976,1.743559,896259253,4075.929487,4.199236e+08,5.415014e+13,2.776008e+07,1.269166e+06,852484.0,635845.0,2.336440e+09,19489162.50,5.208674e+11
1409,2023-09-19,26763.76,1.483430e+08,512111.211644,1.671013,896896158,3553.143885,3.856776e+08,5.581661e+13,2.510024e+07,1.282769e+06,741822.0,493887.0,2.555774e+09,19490050.00,5.239900e+11
1410,2023-09-20,27222.12,1.424000e+08,512343.247832,1.656908,897387893,3779.335616,4.145589e+08,5.711987e+13,2.606572e+07,1.286585e+06,823616.0,551783.0,2.181950e+09,19490931.25,5.303677e+11


In [61]:
# Check nan values
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [62]:
# Check duplicated rows
len(all_data['timestamp'].unique())

1412

In [63]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2019-09-23,10033.05,5.950181e+07,241221.519675,1.243720,457899079,2527.622807,6.738365e+07,1.189059e+13,1.464718e+07,3.220338e+05,544681.0,288149.0,8.550738e+08,17952050.00,1.799873e+11
1,2019-09-24,9683.38,1.200016e+08,241363.197868,1.163618,458186764,2250.082803,9.280029e+07,1.189059e+13,1.922035e+07,3.687073e+05,596954.0,353263.0,3.139283e+09,17953837.50,1.746190e+11
2,2019-09-25,8553.61,4.914318e+08,241545.985674,1.104534,458540193,2229.147436,9.220921e+07,1.189059e+13,1.722270e+07,3.652339e+05,571599.0,347747.0,2.087385e+09,17955612.50,1.545978e+11
3,2019-09-26,8432.23,2.690300e+08,241718.046369,0.970738,458888070,2080.555556,1.021557e+08,1.201767e+13,1.814204e+07,2.732286e+05,554702.0,355775.0,1.313285e+09,17959175.00,1.453526e+11
4,2019-09-27,8055.64,2.863395e+08,241883.747626,1.095424,459242846,2302.295455,8.372682e+07,1.275982e+13,1.373193e+07,2.806025e+05,532292.0,303903.0,1.158430e+09,17960950.00,1.444420e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,2023-09-17,26568.65,3.256705e+07,511582.358862,1.722461,895646989,4102.147651,4.010809e+08,5.415014e+13,2.547981e+07,7.516007e+05,854606.0,611220.0,8.873346e+08,19488275.00,5.186220e+11
1408,2023-09-18,26536.54,2.612496e+07,511839.128976,1.743559,896259253,4075.929487,4.199236e+08,5.415014e+13,2.776008e+07,1.269166e+06,852484.0,635845.0,2.336440e+09,19489162.50,5.208674e+11
1409,2023-09-19,26763.76,1.483430e+08,512111.211644,1.671013,896896158,3553.143885,3.856776e+08,5.581661e+13,2.510024e+07,1.282769e+06,741822.0,493887.0,2.555774e+09,19490050.00,5.239900e+11
1410,2023-09-20,27222.12,1.424000e+08,512343.247832,1.656908,897387893,3779.335616,4.145589e+08,5.711987e+13,2.606572e+07,1.286585e+06,823616.0,551783.0,2.181950e+09,19490931.25,5.303677e+11


In [64]:
# Reorder colunmns
new_columns = ['timestamp', 'market-price', 'total-bitcoins', 'market-cap'] + [col for col in all_data.columns if col not in ['timestamp', 'market-price', 'total-bitcoins', 'market-cap']]
all_data = all_data.reindex(columns=new_columns)
all_data

,timestamp,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2019-09-23,10033.05,17952050.00,1.799873e+11,5.950181e+07,241221.519675,1.243720,457899079,2527.622807,6.738365e+07,1.189059e+13,1.464718e+07,3.220338e+05,544681.0,288149.0,8.550738e+08
1,2019-09-24,9683.38,17953837.50,1.746190e+11,1.200016e+08,241363.197868,1.163618,458186764,2250.082803,9.280029e+07,1.189059e+13,1.922035e+07,3.687073e+05,596954.0,353263.0,3.139283e+09
2,2019-09-25,8553.61,17955612.50,1.545978e+11,4.914318e+08,241545.985674,1.104534,458540193,2229.147436,9.220921e+07,1.189059e+13,1.722270e+07,3.652339e+05,571599.0,347747.0,2.087385e+09
3,2019-09-26,8432.23,17959175.00,1.453526e+11,2.690300e+08,241718.046369,0.970738,458888070,2080.555556,1.021557e+08,1.201767e+13,1.814204e+07,2.732286e+05,554702.0,355775.0,1.313285e+09
4,2019-09-27,8055.64,17960950.00,1.444420e+11,2.863395e+08,241883.747626,1.095424,459242846,2302.295455,8.372682e+07,1.275982e+13,1.373193e+07,2.806025e+05,532292.0,303903.0,1.158430e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,2023-09-17,26568.65,19488275.00,5.186220e+11,3.256705e+07,511582.358862,1.722461,895646989,4102.147651,4.010809e+08,5.415014e+13,2.547981e+07,7.516007e+05,854606.0,611220.0,8.873346e+08
1408,2023-09-18,26536.54,19489162.50,5.208674e+11,2.612496e+07,511839.128976,1.743559,896259253,4075.929487,4.199236e+08,5.415014e+13,2.776008e+07,1.269166e+06,852484.0,635845.0,2.336440e+09
1409,2023-09-19,26763.76,19490050.00,5.239900e+11,1.483430e+08,512111.211644,1.671013,896896158,3553.143885,3.856776e+08,5.581661e+13,2.510024e+07,1.282769e+06,741822.0,493887.0,2.555774e+09
1410,2023-09-20,27222.12,19490931.25,5.303677e+11,1.424000e+08,512343.247832,1.656908,897387893,3779.335616,4.145589e+08,5.711987e+13,2.606572e+07,1.286585e+06,823616.0,551783.0,2.181950e+09



Once we have the daily dataset we will sample it at a frequency of 15 minutes (15T) using the resample method.

This means that the data will be organized in 15-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the dataset by estimating missing values based on the surrounding known values.

In [65]:
# Upsampling to 15min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_15m = all_data.resample('15T').interpolate()
all_data_15m

,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2019-09-23 00:00:00,10033.050000,1.795205e+07,1.799873e+11,5.950181e+07,241221.519675,1.243720,4.578991e+08,2527.622807,6.738365e+07,1.189059e+13,1.464718e+07,3.220338e+05,544681.000000,288149.000000,8.550738e+08
2019-09-23 00:15:00,10029.407604,1.795207e+07,1.799313e+11,6.013201e+07,241222.995490,1.242885,4.579021e+08,2524.731765,6.764841e+07,1.189059e+13,1.469482e+07,3.225200e+05,545225.510417,288827.270833,8.788676e+08
2019-09-23 00:30:00,10025.765208,1.795209e+07,1.798754e+11,6.076222e+07,241224.471304,1.242051,4.579051e+08,2521.840724,6.791317e+07,1.189059e+13,1.474246e+07,3.230062e+05,545770.020833,289505.541667,9.026615e+08
2019-09-23 00:45:00,10022.122812,1.795211e+07,1.798195e+11,6.139242e+07,241225.947119,1.241216,4.579081e+08,2518.949682,6.817792e+07,1.189059e+13,1.479010e+07,3.234923e+05,546314.531250,290183.812500,9.264553e+08
2019-09-23 01:00:00,10018.480417,1.795212e+07,1.797636e+11,6.202263e+07,241227.422933,1.240382,4.579111e+08,2516.058640,6.844268e+07,1.189059e+13,1.483773e+07,3.239785e+05,546859.041667,290862.083333,9.502492e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20 23:00:00,27137.240417,1.949178e+07,5.221530e+11,1.325944e+08,512575.426911,1.641912,8.979184e+08,3154.277873,3.927898e+08,5.711987e+13,2.445131e+07,1.269124e+06,765668.458333,436550.125000,2.519877e+09
2023-09-20 23:15:00,27136.317812,1.949178e+07,5.220637e+11,1.324878e+08,512577.950597,1.641749,8.979241e+08,3147.483767,3.925532e+08,5.711987e+13,2.443376e+07,1.268934e+06,765038.593750,435297.593750,2.523550e+09
2023-09-20 23:30:00,27135.395208,1.949179e+07,5.219744e+11,1.323812e+08,512580.474283,1.641586,8.979299e+08,3140.689661,3.923165e+08,5.711987e+13,2.441621e+07,1.268744e+06,764408.729167,434045.062500,2.527223e+09


# Saving dataset

In [66]:
# Save the 15m dataset
all_data_15m.to_parquet(DATASET_RAW)